# Fortsatt modellering i jakten på de sista tusendelarna accuracy

Nu är jag riktigt nära med 0.9916 i accuracy och kanske nära gränsen för vad som är möjligt med de sämsta bilderna i underlaget. 

Men lite till kan vi försöka hitta.


## Importera nödvändiga paket.

In [46]:
# Av eget intresse vill jag gärna veta hur lång tid olika saker tar.
import time
notebook_start = time.time()  
t0 = time.time()

# Skippa varningar
import warnings

# Boosta prestandan
from sklearnex import patch_sklearn
patch_sklearn()

# Paket för datahantering
import numpy as np
import pandas as pd
import os
import math

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split #, GridSearchCV, cross_val_score

# Preprocessing/pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.kernel_approximation import Nystroem

# Dimensionsreducering
from sklearn.decomposition import PCA

# Feature-bearbetning
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

# Data augmentation och förbehandling - stabil version för SciPy
import scipy.ndimage as ndimage

# Bildbehandling
from PIL import Image

# Modeller 
from sklearn.svm import SVC, LinearSVC

# För export av modellen/scalern för vidare användning i Streamlit-appen
import joblib
from joblib import Parallel, delayed

# Slutrapport
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Tid för denna cell: 0.0 sekunder
>>> Total tid sedan start: 0 minuter och 0 sekunder


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


## Läs in MNISt-datasetet och splitta det. 

*//Best practice: splitta ut testsetet direkt//*

In [33]:
t0 = time.time()

# Läs in alla MNIST-data
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splitta (80/20) med stratifiering för jämna klassfördelningar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# Normalisering (en riktigt bra grej för både PCA och SVC!)
# Genom att dela med 255.0 blir alla värden mellan 0 och 1
X_train = X_train / 255.0
X_test = X_test / 255.0

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print("-" * 40)
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

----------------------------------------
>>> Tid för denna cell: 2.4 sekunder
>>> Total tid sedan start: 0 minuter och 2 sekunder


HoG ska boosta accuracy!

In [34]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def local_hog(img_flat):
            return hog(img_flat.reshape(28, 28), 
                       orientations=self.orientations, 
                       pixels_per_cell=self.pixels_per_cell, 
                       cells_per_block=self.cells_per_block,
                       visualize=False)
        
        return np.array([local_hog(x) for x in X])

Definitioner

In [35]:
t0 = time.time()

def deskew(image):
    # Räta upp lutande siffror 
    img = image.reshape(28, 28)
    
    # Skapa koordinat-matriser (y för rader, x för kolumner)
    y, x = np.mgrid[:28, :28]
    
    # Hitta tyngdpunkten (Center of Mass) med ndimage
    mu = ndimage.center_of_mass(img)
    if np.isnan(mu).any(): # Om bilden är tom
        return img.flatten()
    
    # Beräkna centrala moments (mu11 = kovarians, mu02 = varians i y-led)
    mu11 = np.sum((x - mu[1]) * (y - mu[0]) * img)
    mu02 = np.sum((y - mu[0])**2 * img)
    
    # Om variansen är för liten lutar siffran inte eller är för tunn
    if abs(mu02) < 1e-2:
        return img.flatten()
    
    # Skew-faktorn (förskjutning av x per enhet y)
    skew = mu11 / mu02
    
    # Här rätas x upp genom subtrahering av skew * y
    # Matrisen blir [[1, 0], [skew, 1]] med SciPys omvända ordning.
    matrix = np.array([[1, 0], [skew, 1]])
    
    # Offset för att rotera/skeva kring bildens centrum (14, 14)
    center = np.array([14, 14])
    offset = center - np.dot(matrix, center)
    
    # Transformation
    img_deskewed = ndimage.affine_transform(img, matrix, offset=offset, order=1, mode='constant', cval=0)
    return img_deskewed.flatten()

def shift_image(image, dx, dy):
    return ndimage.shift(image.reshape(28, 28), [dy, dx], cval=0, mode="constant").flatten()

def rotate_image(image, angle):
    return ndimage.rotate(image.reshape(28, 28), angle, reshape=False, cval=0, mode="constant").flatten()

def zoom_image(image, factor):
    rescaled = ndimage.zoom(image.reshape(28, 28), factor)
    if factor > 1.0: # Zooma in (klipp)
        start = int((rescaled.shape[0] - 28) / 2)
        final = rescaled[start:start+28, start:start+28]
    else: # Zooma ut (padda)
        pad = int((28 - rescaled.shape[0]) / 2)
        final = np.pad(rescaled, ((pad, 28-rescaled.shape[0]-pad), (pad, 28-rescaled.shape[1]-pad)), mode='constant')
    return final.flatten()

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Tid för denna cell: 0.0 sekunder
>>> Total tid sedan start: 0 minuter och 2 sekunder


Här kommer lite extra data

In [36]:
t0 = time.time()

# Sökvägar
paths = {
    'custom': 'collected_data',
    'errors': 'mnist_errors'
}

X_extra = []
y_extra = []

def process_and_add(folder, label_pos, split_char='_'):
    added_count = 0
    if not os.path.exists(folder):
        print(f"VARNING: Mappen '{folder}' hittades inte!")
        return 0
    
    files = [f for f in os.listdir(folder) if f.endswith(".png")]
    print(f"Hittade {len(files)} st .png-filer i {folder}...")

    for filename in files:
        try:
            # Extrahera label baserat på filnamnsstruktur
            parts = filename.split(split_char)
            # Hitta siffran i filnamnet
            label = None
            for p in parts:
                if p.isdigit() and len(p) == 1:
                    label = int(p)
                    break
            
            if label is None:
                # Fallback
                label = int(parts[label_pos])
            
            # Bearbeta bilden till MNIST-format
            img = Image.open(os.path.join(folder, filename)).convert('L')
            img = img.resize((28, 28), Image.Resampling.LANCZOS)
            img_array = np.array(img).astype(np.float32) / 255.0
            
            # MNIST-standard: Vit siffra på svart bakgrund
            # Kolla medianen av kanterna
            edge_median = np.median(np.concatenate([img_array[0,:], img_array[-1,:], img_array[:,0], img_array[:,-1]]))
            # Ljus bakgrund -> Invertera
            if edge_median > 0.5: 
                img_array = 1.0 - img_array
                
            X_extra.append(img_array.flatten())
            y_extra.append(label)
            added_count += 1
        except Exception as e:
            print(f"Skippar {filename}: {e}")
            
    return added_count

# Läs in användarskapade bilder
count_c = process_and_add(paths['custom'], label_pos=1)

# Läs in de sämsta bilderna
count_e = process_and_add(paths['errors'], label_pos=1)

if len(X_extra) > 0:
    X_train = np.vstack([X_train, np.array(X_extra)])
    y_train = np.concatenate([y_train, np.array(y_extra)])
    print("-" * 30)
    print(f"SUCCESS: Inkluderat {count_c} egna bilder och {count_e} svåra specialfall.")
    print(f"Nytt antal i X_train: {len(X_train)} (ökning med {len(X_extra)})")
else:
    print("-" * 30)
    print("INGA EXTRA BILDER TILLGÅNGNA. Kontrollera mapparna!")

# Uppdatera X_train_deskewed med alla bilder
print("Rätar upp det nya kombinerade träningssetet (Deskewing)...")
X_train_deskewed = np.array([deskew(img) for img in X_train])

cell_time = time.time() - t0
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")

Hittade 18 st .png-filer i collected_data...
Hittade 50 st .png-filer i mnist_errors...
------------------------------
SUCCESS: Inkluderat 18 egna bilder och 50 svåra specialfall.
Nytt antal i X_train: 56068 (ökning med 68)
Rätar upp det nya kombinerade träningssetet (Deskewing)...
>>> Tid för denna cell: 4.6 sekunder


# Augmentation av data

In [37]:
t0 = time.time()

X_train_deskewed = np.array([deskew(img) for img in X_train])

X_train_augmented = [image for image in X_train_deskewed]
y_train_augmented = [label for label in y_train]

# Det blir en riktig boost med ultra-augmentering och 10 varianter per bild.
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train_deskewed, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

for angle in (-8, -4, 4, 8):
    for image, label in zip(X_train_deskewed, y_train):
        X_train_augmented.append(rotate_image(image, angle))
        y_train_augmented.append(label)

for factor in (0.9, 1.1):
    for image, label in zip(X_train_deskewed, y_train):
        X_train_augmented.append(zoom_image(image, factor))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print("-" * 40)
print(f"Ny datamängd: {len(X_train_augmented)} rader")
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

----------------------------------------
Ny datamängd: 616748 rader
>>> Tid för denna cell: 41.6 sekunder
>>> Total tid sedan start: 0 minuter och 51 sekunder


Preparera testsetet så att jämförelsen blir rättvis.

In [38]:
t0 = time.time()

# Här rätas testbilderna upp för att matcha modellen
X_test_deskewed = np.array([deskew(img) for img in X_test])

X_test_deskewed = X_test_deskewed.astype('float32')

cell_time = time.time() - t0
print("-" * 40)
print(f">>> Sökningen klar på {cell_time:.1f} sekunder.")

----------------------------------------
>>> Sökningen klar på 1.2 sekunder.


## Dags att skapa en pipeline som används i hela projektet, en "single source of truth". 

In [39]:
t0 = time.time()

# Detta är vår "Ultra-Pipeline"
# Vi kombinerar Notebook 3:s PCA med form-analys från HOG
final_pipe_ultra = Pipeline([
    ('feature', FeatureUnion([
        ('pca', PCA(n_components=140)), # Från Notebook 3
        ('hog', HogTransformer())      # Vår nya form-förstärkare
    ])),
    ('scaler', StandardScaler()),      # Livsviktig för att balansera PCA och HOG
    ('svc', SVC(
        C=25, 
        kernel='rbf', 
        gamma='scale', 
        cache_size=2000,              # Utnyttjar dina 32GB RAM för snabbare träning
        probability=False, 
        verbose=True
    ))
])

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Pipeline definierad! Tid: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Pipeline definierad! Tid: 0.0 sekunder
>>> Total tid sedan start: 1 minuter och 24 sekunder


In [40]:
# Importera Garbage Collector för minneshanteringen
import gc 

t0 = time.time()

# 1. Konvertera till float32 direkt för att spara hälften av RAM
print(f"Konverterar {len(X_train_augmented)} rader till float32...")
X_train_final = X_train_augmented.astype('float32')
y_train_final = y_train_augmented

# 2. RENSA MINNET AGGRESSIVT
# Vi raderar originalet direkt eftersom vi nu har 'X_train_final'
del X_train_augmented
gc.collect() 

# 3. Uppdatera Pipelinen för maxad precision (C=25 och mer PCA)
final_pipe_ultra.set_params(
    feature__pca__n_components=140, 
    svc__C=25, 
    svc__cache_size=2000, # Låg cache för att undvika minneskrockar vid 600k rader
    svc__verbose=True
)

print(f"STARTAR FULLSKALIG TRÄNING: {len(X_train_final)} rader.")
print("Detta är gränsen för vad en vanlig PC klarar. Beräknar...")

try:
    final_pipe_ultra.fit(X_train_final, y_train_final)
    print("SUCCESS! Modellen tränad på ALLA rader.")
except MemoryError:
    print("AI-Haveri: 600k var för mycket för RBF-kerneln. Kör på 250k istället.")

# 4. Snabbkoll på Accuracy (utan TTA)
base_accuracy = final_pipe_ultra.score(X_test_deskewed, y_test)
print("\n" + "="*30)
print(f"BAS-ACCURACY (600k rader): {base_accuracy:.5f}")
print("="*30)

cell_time = time.time() - t0
print(f">>> Full träning klar på: {cell_time/60:.1f} minuter")

Konverterar 616748 rader till float32...
STARTAR FULLSKALIG TRÄNING: 616748 rader.
Detta är gränsen för vad en vanlig PC klarar. Beräknar...
SUCCESS! Modellen tränad på ALLA rader.

BAS-ACCURACY (600k rader): 0.99357
>>> Full träning klar på: 7.3 minuter


Dumpa ned modellen så den finns kvar även om det kraschar

In [ ]:
joblib.dump(final_pipe_ultra, 'mnist_svc_hog_max.joblib')